In [1]:
print("ok")

ok


In [2]:
from typing import Annotated
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END,StateGraph, START
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

In [3]:
import os
from dotenv import load_dotenv # Install with: pip install python-dotenv

load_dotenv() 
api_key = os.getenv("TAVILY_API_KEY")
print(f"Key loaded: {api_key}") # Verify it works without printing the whole key

Key loaded: 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


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.5-pro') # Also correcting model name to a likely intended valid one
response = llm.invoke("what is capital of USA?")
print(response.content)
gemini_model = llm # Keep for downstream use

The capital of the USA is **Washington, D.C.**


In [5]:
tavaily_tool=TavilySearchResults()

C:\Users\pc\AppData\Local\Temp\ipykernel_9948\820692609.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavaily_tool=TavilySearchResults()


In [6]:
tavaily_tool.invoke("what is a gdp?")

[{'title': 'What is GDP? - YouTube',
  'url': 'https://www.youtube.com/watch?v=_pTkwg6iUV0',
  'content': "[Music] gdp is the best-known three-letter acronym in economics but what exactly is gdp gdp or gross domestic product is the total market value of all final goods and services produced in an economy in a given year it measures the production of goods like pizza new houses cell phones and services like your favorite streaming channels the calculations are simple accounting multiplying the prices of the goods and services by the quantity sold and adding it all up for the entire economy it's important to note that all these goods are measured in the price at which they are sold to the end consumer in u.s dollars because one person's spending is another person's income gdp can be tracked by either adding up income earned from selling goods and services or by tracking spending on goods and",
  'score': 0.882276},
 {'title': 'What Is Gross Domestic Product (GDP)?',
  'url': 'https://mru

In [ ]:
import asyncio
import json
import re
from typing import Annotated
from pydantic import BaseModel, Field
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI


In [60]:
@tool
def news_vector_db(query: Annotated[str, "News query"]):
    """Retrieve latest relevant news for a stock."""
    return "This is the News data"


@tool
def financial_data(query: Annotated[str, "Financial query"]):
    """Retrieve latest financial fundamentals for a stock."""
    return "This is the Financial data"


@tool
def historical_model(query: Annotated[str, "OHLCV data"]):
    """Predict next-minute movement using historical OHLCV indicators."""
    return "This is the Historical model output"


@tool
def market_live_data(query: Annotated[str, "Live market query"]):
    """Retrieve live market technical data for a stock."""
    return "This is the Market live data"


@tool
def book_data(query: Annotated[str, "Book query"]):
    """Retrieve relevant stock market book knowledge."""
    return "This is the Book data"


In [62]:
import re
import json

with open("signal.txt", "r", encoding="utf-8") as file:
    raw = file.read()

# 1️⃣ Remove newlines (optional)
raw = raw.strip()

# 2️⃣ Replace single quotes with double quotes
raw = raw.replace("'", '"')

# 3️⃣ Quote unquoted keys
# symbol: → "symbol":
raw = re.sub(r'([{,]\s*)(\w+)\s*:', r'\1"\2":', raw)

# 4️⃣ Quote ISO datetime values if not quoted
# 2026-02-13T15:59:32.000Z → "2026-02-13T15:59:32.000Z"
raw = re.sub(
    r':\s*(\d{4}-\d{2}-\d{2}T[0-9:\.\-Z]+)',
    r': "\1"',
    raw
)

# 5️⃣ Replace undefined with null
raw = raw.replace("undefined", "null")

# 6️⃣ Remove trailing commas before } or ]
raw = re.sub(r',\s*([}\]])', r'\1', raw)

# Now parse
signal = json.loads(raw)

print("Parsed Signal:", signal)


Parsed Signal: {'exch': 'NSE', 'instrument': 'EQUITY', 'securityId': '2885', 'symbolName': 'RELIANCE INDUSTRIES LTD', 'underlyingSymbol': 'RELIANCE', 'displayName': 'RELIANCE INDUSTRIES', 'lotSize': 1, 'tickSize': 10, 'ltp': 1419.5999755859375, 'strikeGap': 10, 'volume': 10755693, 'oi': 0, 'open': 1445.5, 'high': 1450.699951171875, 'low': 1416.300048828125, 'close': 1419.6, 'depth': [{'bidQty': 459, 'askQty': 0, 'bidOrders': 5, 'askOrders': 0, 'bidPrice': 1419.5999755859375, 'askPrice': 0}, {'bidQty': 0, 'askQty': 0, 'bidOrders': 0, 'askOrders': 0, 'bidPrice': 0, 'askPrice': 0}, {'bidQty': 0, 'askQty': 0, 'bidOrders': 0, 'askOrders': 0, 'bidPrice': 0, 'askPrice': 0}, {'bidQty': 0, 'askQty': 0, 'bidOrders': 0, 'askOrders': 0, 'bidPrice': 0, 'askPrice': 0}, {'bidQty': 0, 'askQty': 0, 'bidOrders': 0, 'askOrders': 0, 'bidPrice': 0, 'askPrice': 0}], 'change': -1.7197846160221562e-06, 'ltq': 3, 'ltt': 1770998372, 'lttDate': '2026-02-13T15:59:32.000Z', 'atp': 1423.72998046875, 'totalBuyQty': 

In [63]:
class DecisionOutput(BaseModel):
    decision: str = Field(description="BUY or SELL")
    confidence: float = Field(description="Confidence between 0 and 1")
    reason: str = Field(description="Concise evidence-based reasoning")

structured_llm = llm.with_structured_output(DecisionOutput)


In [64]:
async def collect_data(signal):

    query = f"Analyse stock {signal}"

    results = await asyncio.gather(
        news_vector_db.ainvoke(query),
        financial_data.ainvoke(query),
        historical_model.ainvoke(signal),
        market_live_data.ainvoke(query),
        book_data.ainvoke(query),
    )

    return {
        "news": results[0],
        "financial": results[1],
        "historical": results[2],
        "live_market": results[3],
        "book": results[4],
    }


In [65]:
async def make_decision(signal):

    data = await collect_data(signal)

    prompt = f"""
    Trading decision engine.
    Evaluate signal and market intelligence.

    Signal:
    {signal}

    Market Data:
    {data}

    Return:
    - BUY or SELL
    - Confidence (0-1)
    - Concise evidence reasoning
    JSON only.
    """

    result = await structured_llm.ainvoke(prompt)

    return result


In [66]:
async def main():

    decision = await make_decision(signal)

    print("Final Decision:")
    print(decision.model_dump_json(indent=2))

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop